In [1]:
!pip install pyomo


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.5 MB/s eta 0:00:00


In [26]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 122400 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.11.4+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.108.6+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-2_amd64.deb ...
Unpacking coinor-libcgl1 (0.60.3+repack1-2) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.10.3+repack1-1build1_amd64.deb ...
Unpacking coinor-libcbc3 (2.10.3+repack1-1build1) ...
Sel

In [1]:
days = [[6,9,9,8,3,3,7,8,8,5,3,3,2],
        [6,10,7,7,3,4,7,5,9,5,3,4,3],
        [7,9,9,6,3,4,6,8,7,4,3,3,3],
        [6,9,8,6,4,4,5,8,7,5,4,3,4],
        [6,7,8,7,3,5,6,7,6,5,3,3,3],
        [6,9,9,4,3,3,4,5,5,5,3,3,2],
        [5,7,6,5,4,3,4,5,6,5,3,3,3]]
shifts=[[1,1,1,1,0,1,1,1,1,0,0,0,0],#C1
        [0,1,1,1,0,1,1,1,1,1,0,0,0],#C2
        [0,0,1,1,1,0,1,1,1,1,1,0,0],#C3
        [0,0,0,1,1,1,1,1,0,1,1,1,0],#C4
        [0,0,0,0,1,1,1,1,1,0,1,1,1],#C5
        [1,1,1,1,0,0,0,0,0,1,1,1,1]]#C6

#Simplex method

In [71]:
#Q1
import pyomo.environ as pyo
from functools import reduce
no_shifts = len(shifts)
ncs = []
cas = [0 for i in range(no_shifts)]

for i in range(len(days)):
  model = pyo.ConcreteModel()
  
  vars = list(range(0,no_shifts))

  model.x = pyo.Var(vars,domain=pyo.NonNegativeIntegers)
  model.c = pyo.ConstraintList()
  for j in range(len(days[0])):
    temp = None
    for k in range(no_shifts):
      if temp is None:
        temp = model.x[vars[k]]*shifts[k][j]
      else:
        temp += model.x[vars[k]]*shifts[k][j]
    model.c.add(temp >= days[i][j])
  model.objective = pyo.Objective(expr = reduce(lambda a, b: a + model.x[b], vars, 0),sense=pyo.minimize)
  solver = pyo.SolverFactory('cbc')
  result = solver.solve(model)
  print("Day: "+str(i) + " - " + str(model.objective())+" CSR")
  ncs.append(model.objective())
  for j in vars:
    cas[j]+=model.x[j]()
    print("C"+str(j+1)+": "+str(model.x[j]()))


Day: 0 - 12.0 CSR
C1: 2.0
C2: 3.0
C3: 0.0
C4: 0.0
C5: 3.0
C6: 4.0
Day: 1 - 13.0 CSR
C1: 2.0
C2: 4.0
C3: 0.0
C4: 0.0
C5: 3.0
C6: 4.0
Day: 2 - 12.0 CSR
C1: 3.0
C2: 2.0
C3: 0.0
C4: 1.0
C5: 2.0
C6: 4.0
Day: 3 - 13.0 CSR
C1: 1.0
C2: 3.0
C3: 0.0
C4: 1.0
C5: 3.0
C6: 5.0
Day: 4 - 10.0 CSR
C1: 3.0
C2: 1.0
C3: 1.0
C4: 1.0
C5: 1.0
C6: 3.0
Day: 5 - 12.0 CSR
C1: 0.0
C2: 3.0
C3: 0.0
C4: 1.0
C5: 2.0
C6: 6.0
Day: 6 - 11.0 CSR
C1: 0.0
C2: 2.0
C3: 0.0
C4: 0.0
C5: 4.0
C6: 5.0


In [3]:
#Q2


nd = 7

# for i in range(len(days)):
model = pyo.ConcreteModel()
# vars = list(range(0,no_shifts))

model.x = pyo.Var(domain=pyo.NonNegativeIntegers)
model.c = pyo.ConstraintList()
  
model.c.add(nd*max(ncs)-sum(ncs) + (nd*model.x) >= model.x+max(ncs))
# print(reduce(lambda a, b: a + model.x[b], vars,0))
model.objective = pyo.Objective(expr = model.x,sense=pyo.minimize)
solver = pyo.SolverFactory('cbc')
result = solver.solve(model)
print("Number of CSR needed: " + str(model.objective()))
print(model.x())
nc = model.objective()+max(ncs)
# model.Constraint1 = pyo.Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)

Number of CSR needed: 1.0
1.0


In [98]:
#Q3
import math


model = pyo.ConcreteModel()
vars = list(range(int(nc*len(days)*no_shifts)))

model.x = pyo.Var(vars,domain=pyo.Binary)
model.c = pyo.ConstraintList()

for i in range(int(nc)):
  temp2 = None
  for j in range(len(days)):
    temp1 = None
    for k in range(no_shifts):
      if temp1 is None:
        temp1 = model.x[i*(len(days)*no_shifts)+j*(len(days)-1)+k]
      else:
        temp1 += model.x[i*(len(days)*no_shifts)+j*(len(days)-1)+k]
      if temp2 is None:
        temp2 = model.x[i*(len(days)*no_shifts)+j*(len(days)-1)+k]
      else:
        temp2 += model.x[i*(len(days)*no_shifts)+j*(len(days)-1)+k]
    model.c.add(temp1 <= 1)
    # print(temp1)
  model.c.add(temp2 <= nd-1)

for j in range(len(days)):
  for u in range(len(days[0])):
    temp_3 = None
    for k in range(no_shifts):  
        for i in range(int(nc)):
          if temp_3 is None:
            temp_3  = model.x[i*(len(days)*no_shifts)+j*(len(days)-1)+k]*shifts[k][u]
          else:
            temp_3  += model.x[i*(len(days)*no_shifts)+j*(len(days)-1)+k]*shifts[k][u]
    model.c.add(temp_3 >= days[j][u])

for k in range(no_shifts):
  # print(cas[k]/nc)
  lb = math.floor(cas[k]/nc)
  ub = math.ceil(cas[k]/nc)
  
  for i in range(int(nc)):
    temp_4 = None
    for j in range(len(days)):
      if temp_4 is None:
        temp_4 = model.x[i*(len(days)*no_shifts)+j*(len(days)-1)+k]
      else:
        temp_4 += model.x[i*(len(days)*no_shifts)+j*(len(days)-1)+k]

    model.c.add(temp_4 >= lb)
    model.c.add(temp_4 <= ub)


model.objective = pyo.Objective(expr = reduce(lambda a, b: a + model.x[b], vars, 0),sense=pyo.minimize)
solver = pyo.SolverFactory('cbc')
result = solver.solve(model)
print(result)
print("Objective: " + str(model.objective()))
# s=0

counts = [0 for i in range(no_shifts+1)]

for i in range(int(nc)):
  for j in range(len(days)):
    s=-1
    for k in range(no_shifts):
      if model.x[i*(len(days)*no_shifts)+j*(len(days)-1)+k]()==1.0:
        s = k
    counts[s+1]+=1
    print(s+1,end=' ')
  print()



Problem: 
- Name: unknown
  Lower bound: 83.0
  Upper bound: 83.0
  Number of objectives: 1
  Number of constraints: 322
  Number of variables: 588
  Number of binary variables: 588
  Number of integer variables: 588
  Number of nonzeros: 588
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.05
  Wallclock time: 0.05
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.06833624839782715
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objective: 83.0
0 6 1 5 6 2 4 
6 5 6 3 1 2 0 
2 6 1 2 5 0 6 
3 2 6 1 0 6 5 
0 1 2 6 0 5 6 
2 6 5 4 1 6 0 
3 2 6 0 5 6 1 
6 2 0 1 3 6 5 
5 1 6 6 0 2 2 
1 5 4 6 6 0 2 
6 2 0 5 1 3 6 
1 6 2 2 6 5 0 
6 0 1 2 2 6 5 
2 5 3 6 0 6 6 


In [102]:
[i/nc for i in counts[1:]]

[0.9285714285714286,
 1.2857142857142858,
 0.42857142857142855,
 0.21428571428571427,
 1.0,
 2.0714285714285716]

In [103]:
[i/nc for i in cas]

[0.7857142857142857,
 1.2857142857142858,
 0.07142857142857142,
 0.2857142857142857,
 1.2857142857142858,
 2.2142857142857144]